<a href="https://colab.research.google.com/github/acts701/LLM/blob/main/250118_%EA%B0%9C%EB%B0%9C%EC%9D%BC%EC%A7%80_vectordb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 번역 후 Pinecone
- 국문/영문의 문제가 아니었나보다.
- model도 jhgan/ko-sbert-multitask로 바꿔봤는데 여전히 search 안됨
```python
model = SentenceTransformer("jhgan/ko-sbert-multitask")
query_vector = model.encode(translated_query).tolist()
search_results = index.query(
        vector=query_vector,
        top_k=top_k,
        namespace="ns1",
        include_metadata=True
    )
query_text = "성경이 말하는 예언 선지자들의 말씀은"
검색 결과:
ID: SA_1_21_31.txt-5, Score: 0.65489459, Original Text: 그러므로 주 만군의 여호와 이스라엘의 전능자가 말씀하시되 슬프다., Translated Text: Therefore, the Lord, the God of Hosts, the Mighty One of Israel, declares, "Woe to you."
ID: SA_1_21_31.txt-8, Score: 0.620340824, Original Text: 그러나 패악한 자와 죄인은 함께 패망하고 여호와를 버린 자도 멸망할 것이라., Translated Text: However, the wicked and sinners will perish together, and those who forsake the Lord will be destroyed.
ID: SA_1_1_9.txt-3, Score: 0.609233916, Original Text: 유다왕 우시아와 요담과 히스기야 시대 아모스의 아들 이사야가 유다와 예루살렘에 대하여 본 이상이라 하늘이여 드리라 땅이여 귀를 기울이라 여호와께서 말씀하시기를 내가 자식을 양육하였거늘 그들이 나를 거역하였도다, Translated Text: During the reigns of Uzziah, Jotham, Ahaz, and Hezekiah, kings of Judah, the son of Amos, Isaiah, saw visions concerning Judah and Jerusalem. Hear, O heavens! Listen, O earth! For the Lord has spoken: "I reared children and brought them up, but they have rebelled against me.
```
- 결과에 영향을 주는 hyper parameter는?
    - model, 국문, 영문, index query 방
- embedding vector를 처음 써서 성능 평가를 이렇게 해도 되는건지도 잘 모르겠음
- 혹시나 영문으로 바로 검색을 해봤다.
    - 내가 사용법을 잘 몰라서 그런지
``` python
query_vector = model.encode("The important themes of Isaiah").tolist()
검색 결과:
ID: SA_1_1_9.txt-2, Score: 0.681410372, Translated Text: Isaiah 1:1
ID: SA_2_1_5.txt-1, Score: 0.61779362,  Translated Text: The word that Isaiah son of Amoz saw concerning Judah and Jerusalem. In the last days the mountain of the Lord's temple will be established as the highest of the mountains.
ID: SA_1_21_31.txt-5, Score: 0.573399603, Translated Text: Therefore, the Lord, the God of Hosts, the Mighty One of Israel, declares, "Woe to you."
```

# FAISS (Facebook AI Similarity Search)
- 오 감격의 facebook!
    - pinecone과 FAISS의 성능 차이까지는 모르겠지만 적어도 내 상황에서는 FAISS가 더 맞다.
    - 아래는 embedding 할 떄 국문과 영문을 함께 저장한 후 search 결과
-
```python
query_text = "성경이 말하는 예언 선지자들의 말씀은"
검색 결과:
Rank 1:
ID: SA_1_1_9.txt-68
Original Text: 성경이 말하는 예언 선지자들의 말씀은 이제 이사야서부터 시작되는 이사야 예레미야 에스겔 다니엘 그리고 열두 선지자들로 얘기하는이 선지서들의 말씀은 지금 당대에 우리 시대에이 사람들 살고 있는이 시대의 하나님의 뜻을 전달하는 것이며 만약에 너희들이 하나님의 뜻을 받지 않으면은 어떻게 어떻게 될 것이다라는
Rank 2:
ID: SA_1_1_9.txt-135
Original Text: 그래서 2절에 보면은 여호와께서 말씀하시기를
 성경에 독특한 말이 이것인데 말씀을 들어야 되는데 말씀을 보았을 때 그것을 이상이라 이런 독특한 단어를 쓰고 있습니다
Rank 3:
ID: SA_2_1_5.txt-36
Original Text: 성경이 놀라운 것은 이미 그러한 시대가 시작되었다라고 얘기하는 데 있습니다.
```
```python
query_text = "이사야의 중요한 주제"
검색 결과:
Rank 1:
ID: SA_1_21_31.txt-100
Original Text: 이러지 이거는 이건 우리가 이사야에서 이사야님이시니까 이렇게 들어주고 있지.
Translated Text: We should listen to what Isaiah is saying because he is Isaiah.
Distance: 164.3745880126953
Rank 2:
ID: SA_1_21_31.txt-38
Original Text: 이사야 후반부의 주제가 벌써 시작되고 있는 것입니다.
Translated Text: The theme of the latter part of Isaiah has already begun.
Distance: 169.1984100341797
Rank 3:
ID: SA_2_1_5.txt-57
Original Text: 이사야의 중요한 주제가 심판과 구원입니다.
Translated Text: The important themes of Isaiah are judgment and salvation.
Distance: 173.57980346679688
```
- 아래는 영문 과정 없이 국문으로만 embedding 하고 조회한 결과
    - 아주 좋은 것은 아니지만 그렇다고 안되는 것도 아님.
``` python
# Bad Case
query_text = "이사야의 중요한 주제"
Rank 1:
ID: SA_2_1_5.txt-70
Original Text: 이사야는 이 점에 대해서 설명을 해 주고 있는 것이에요.
Distance: 207.16464233398438
Rank 2:
ID: SA_1_1_9.txt-13
Original Text: 그래서 이사야서입니다
Distance: 226.4801025390625
Rank 3:
ID: SA_2_1_5.txt-570
Original Text: 이사야가 이걸 본 중요한 배경은 뭐겠습니까?
Distance: 232.17298889160156
```
``` python
# Good Case
query_text = "성령의 시대 교회의 시대"
검색 결과:
Rank 1:
ID: SA_2_1_5.txt-874
Original Text: 지금 이 교회의 시대를 얘기하고 있는 것이 말씀이 나오는 것이죠.
Distance: 201.12875366210938
Rank 2:
ID: SA_2_1_5.txt-145
Original Text: 성경은 그래서 신약 성경은 말리를 성령의 시대 또는 지금 우리 식으로 얘기하면 교회의 시대다 얘기하고 있는데, 우리가 교회 안에 들어와 있고 교회의 신앙 안에 들어와 있는 가 교회를 하나님의 기업으로 받아 누리는 것은 종말을 사는 것입니다.
Distance: 208.82797241210938
Rank 3:
ID: SA_2_1_5.txt-920
Original Text: 통합된 삶 구원으로 통합된 것이 그럴 때 그런 마음으로 사람들이 교회에 모이고 하나님께 기도하고 정말 다시 한 번 예루살렘 그리고 교회를 다시 한 번 만방의 모든 산 위에 우뚝 서서 거기서 말씀의 물들이 쫙 나올 때 이방인들이 쇄도에 들어오는 것 그때를 다시 한 번 우리에게 주시기를 구하는 것입니다.
Distance: 272.29888916015625
```
- 성능 검증 plan을 세워야겠다. colab 환경에서 session을 여러개 유지할 수 없어서 여러 조건에서 성능 비교가 제한적이다. 하나의 session에서 봐야 하는 것들을 최대한 다 보고 넘어가야지 개발 기간이 짧아진다. 집중력도...
- 애매하다. 영문을 꼭 써야 할까?
    - sample 검증 시 100%는 안되지만 어느 정도 성능이 나오니 FAISS & 국문으로 진행하자.